<a href="https://colab.research.google.com/github/mfmarlonferrari/tcc2020recommender/blob/main/KNN_API.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#https://pypi.org/project/mysql-connector/
!pip install mysql-connector flask-ngrok

     |████████████████████████████████| 11.9MB 337kB/s 
  Created wheel for mysql-connector: filename=mysql_connector-2.2.9-cp36-cp36m-linux_x86_64.whl size=247950 sha256=c5ca9aec2dcdeaacdd8512df4fb304f1f837e9044043ec7b344436b60bfc2644
  Stored in directory: /root/.cache/pip/wheels/8c/83/a1/f8b6d4bb1bd6208bbde1608bbfa7557504bed9eaf2ecf8c175
Successfully built mysql-connector


In [2]:
import pandas as pd
import mysql.connector
from sklearn.neighbors import NearestNeighbors
import json

Conexão com a base de Dados transacional

In [12]:
def recomendacao(id_usuario):
  host="database-1.c6nlhkjxpqnn.sa-east-1.rds.amazonaws.com"
  port=3306
  dbname="DB_TCC"
  user="admin"
  password="admin123"

  conn = mysql.connector.connect(user=user, password=password,
                              host=host,
                              database=dbname)

  cur = conn.cursor()

  cur.execute('select * from questions')
  dados = cur.fetchall()
  dataset = pd.DataFrame(dados, columns=['ID', 'USUARIO', 'LIVRO', 'TITULO', 'RATING','D1','D2'])[['USUARIO','LIVRO','TITULO','RATING']]

  print(dataset.shape)

  movies_features = dataset.pivot_table(index='USUARIO', columns='TITULO', values='RATING').fillna(0)

  modelo = NearestNeighbors(metric='cosine', n_neighbors=3)
  modelo.fit(movies_features)

  distancia, indices_vizinhos = modelo.kneighbors(movies_features.loc[id_usuario].values.reshape(1,-1), n_neighbors=4)

  parametros = {}

  for i in range(0, len(distancia.flatten())):
    if movies_features.index[indices_vizinhos.flatten()[i]] != id_usuario:
      parametros[movies_features.index[indices_vizinhos.flatten()[i]]] = distancia.flatten()[i]

  id_recomendado = movies_features.index[indices_vizinhos.flatten()][1]
  base = movies_features.loc[id_usuario].to_frame()
  recomendado = movies_features.loc[id_recomendado].to_frame()

  novos_titulos = pd.merge(base,recomendado,on='TITULO',how='inner')
  novos_titulos = novos_titulos[(novos_titulos[id_recomendado] > 0) & (novos_titulos[id_usuario] == 0)].reset_index()

  livros = json.loads(novos_titulos.to_json(force_ascii=False))
  parametros["livros"] = livros

  resultado_recomendacao = {"parametros":parametros}
  return str(resultado_recomendacao)

In [14]:
recomendacao(8)

(42, 4)


'{\'parametros\': {5: 1.0, 6: 1.0, 7: 1.0, \'livros\': {\'TITULO\': {\'0\': "Harry Potter and the Sorcerer\'s Stone (Harry Potter, #1)", \'1\': \'O Ladrão de Raios (Percy Jackson e os Olimpianos, #1)\', \'2\': \'O Menino Maluquinho\'}, \'8\': {\'0\': 0.0, \'1\': 0.0, \'2\': 0.0}, \'5\': {\'0\': 5.0, \'1\': 3.0, \'2\': 4.0}}}}'

In [15]:
from flask_ngrok import run_with_ngrok
from flask import Flask
app = Flask(__name__)
run_with_ngrok(app)   #starts ngrok when the app is run
@app.route("/recomendar/<idUsuario>")
def recomendar(idUsuario):
    idUsuario = int(idUsuario)
    return recomendacao(idUsuario)
  
app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://23e693eec596.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


127.0.0.1 - - [25/Oct/2020 19:45:38] "GET / HTTP/1.1" 404 -
127.0.0.1 - - [25/Oct/2020 19:45:39] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [25/Oct/2020 19:45:49] "GET /recomendar/8 HTTP/1.1" 200 -


(42, 4)


127.0.0.1 - - [25/Oct/2020 19:46:03] "GET /recomendar/8 HTTP/1.1" 200 -


(42, 4)
